In [296]:
import pandas as pd
import numpy as np
import os

In [297]:
path_to_csv_files= "data"

combined_data = pd.DataFrame()

for file in os.listdir(path_to_csv_files):
  if file.endswith('.csv'):
    file_path = os.path.join(path_to_csv_files,file)
    df_comb = pd.read_csv(file_path,encoding='Latin-1')
    combined_data = pd.concat([combined_data, df_comb])
  


combined_data 



,Text,Label
0,"According to Gran , the company has no plans t...",neutral
1,Technopolis plans to develop in stages an area...,neutral
2,The international electronic industry company ...,negative
3,With the new production plant the company woul...,positive
4,According to the company 's updated strategy f...,positive
...,...,...
51,Sports fandom can foster a sense of community ...,positive
52,Sports events offer a platform for showcasing ...,positive
53,The pressure to win in sports can overshadow t...,negative
54,Sports programs in schools play a crucial role...,positive


In [298]:
df = combined_data.copy()
df

,Text,Label
0,"According to Gran , the company has no plans t...",neutral
1,Technopolis plans to develop in stages an area...,neutral
2,The international electronic industry company ...,negative
3,With the new production plant the company woul...,positive
4,According to the company 's updated strategy f...,positive
...,...,...
51,Sports fandom can foster a sense of community ...,positive
52,Sports events offer a platform for showcasing ...,positive
53,The pressure to win in sports can overshadow t...,negative
54,Sports programs in schools play a crucial role...,positive


In [299]:
df.columns

Index(['Text', 'Label'], dtype='object')

In [300]:

filtered_neutral = df[df.Label=='neutral']

print(filtered_neutral)

                                                   Text    Label
0     According to Gran , the company has no plans t...  neutral
1     Technopolis plans to develop in stages an area...  neutral
58    At the request of Finnish media company Alma M...  neutral
59    In Sweden , Gallerix accumulated SEK denominat...  neutral
60    The company supports its global customers in d...  neutral
...                                                 ...      ...
5833  In 2008 , Kemira recorded revenue of approxima...  neutral
5834  Investments in product development stood at 6....  neutral
5838  Nordic Walking was first used as a summer trai...  neutral
5839  According shipping company Viking Line , the E...  neutral
5840  In the building and home improvement trade , s...  neutral

[6009 rows x 2 columns]


In [301]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [302]:
df.Label

0      neutral
1      neutral
2     negative
3     positive
4     positive
        ...   
51    positive
52    positive
53    negative
54    positive
55    negative
Name: Label, Length: 10897, dtype: object

In [303]:
df.isnull().sum()

Text     0
Label    0
dtype: int64

In [304]:
Le = LabelEncoder()

df.Label = Le.fit_transform(df.Label)

df.Label

0     1
1     1
2     0
3     2
4     2
     ..
51    2
52    2
53    0
54    2
55    0
Name: Label, Length: 10897, dtype: int32

In [305]:
port_stem = PorterStemmer()

In [306]:
df

,Text,Label
0,"According to Gran , the company has no plans t...",1
1,Technopolis plans to develop in stages an area...,1
2,The international electronic industry company ...,0
3,With the new production plant the company woul...,2
4,According to the company 's updated strategy f...,2
...,...,...
51,Sports fandom can foster a sense of community ...,2
52,Sports events offer a platform for showcasing ...,2
53,The pressure to win in sports can overshadow t...,0
54,Sports programs in schools play a crucial role...,2


In [307]:
def stemming(news):
  stemming_content = re.sub('[^a-zA-Z]',' ',news)
  stemming_content = stemming_content.lower()
  stemming_content = stemming_content.split()
  stemming_content = [port_stem.stem(word) for word in stemming_content if not word in stopwords.words('english')]
  stemming_content = ' '.join(stemming_content)
  return stemming_content


df_1 = df.copy()

In [308]:
df_1.Text=df.Text.apply(stemming)

In [309]:
df_1

,Text,Label
0,accord gran compani plan move product russia a...,1
1,technopoli plan develop stage area less squar ...,1
2,intern electron industri compani elcoteq laid ...,0
3,new product plant compani would increas capac ...,2
4,accord compani updat strategi year baswar targ...,2
...,...,...
51,sport fandom foster sens commun belong among fan,2
52,sport event offer platform showcas cultur dive...,2
53,pressur win sport overshadow enjoy play game,0
54,sport program school play crucial role charact...,2


In [310]:
x= df_1.Text.values
y=df_1.Label.values

In [311]:
y.ravel()

array([1, 1, 0, ..., 0, 2, 0])

In [312]:
vect=TfidfVectorizer()
vect.fit(x)
x= vect.transform(x)


In [313]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [314]:
y_train

array([1, 2, 0, ..., 1, 2, 1])

In [351]:



from xgboost import XGBClassifier

model1  = XGBClassifier(n_estimators=800,random_state=42)
model1.fit(x_train,y_train)

pred= model1.predict(x_test)



In [347]:
x.shape

(10897, 8456)

In [352]:
score=accuracy_score(pred,y_test)
score

0.8004587155963303

In [369]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Dense(units=512, activation="relu",input_shape=(x_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(units=3)
])

# Compile the model
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])

# Train the model (ensure x_train and y_train are defined)
model.fit(x_train, y_train, epochs=10)


Epoch 1/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6269 - loss: 0.8123
Epoch 2/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8887 - loss: 0.2813
Epoch 3/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9291 - loss: 0.1603
Epoch 4/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9380 - loss: 0.1297
Epoch 5/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9429 - loss: 0.1154
Epoch 6/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9440 - loss: 0.1076
Epoch 7/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9461 - loss: 0.1027
Epoch 8/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9445 - loss: 0.1055
Epoch 9/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9482 - loss: 0.0991
Epoch 10/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9485 - loss: 0.0938


In [370]:
pred2 = model.predict(x_test)
pred_categorical = np.argmax(pred2,axis=1)
score =accuracy_score(pred_categorical,y_test)

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [371]:
score

0.8472477064220183

In [330]:
import joblib
joblib.dump(model, 'model.pkl')

['model.pkl']

In [331]:
tweets_input = np.array([
    "@BillyM2k 🤣🤣",
    "From the top of Starbase high bay https://t.co/kRtJBrwrqW",
    "RT @cleantechnica: Jay Leno Breaks World Record In Tesla’s New Plaid Model S – “I Love This Car” https://t.co/Rxd7VNIZpx",
    "Stacking Super Heavy Aft Section https://t.co/itydacQ4hM",
    "@MileyCyrus 133T H4X0R got my phone. Nuthin I could do. Sorry babe.",
    "@tobyliiiiiiiii @f_rmnn Cool"
])

df3= pd.DataFrame(data=tweets_input)






In [332]:
df3.columns=['input']

In [333]:
df3

,input
0,@BillyM2k 🤣🤣
1,From the top of Starbase high bay https://t.co...
2,RT @cleantechnica: Jay Leno Breaks World Recor...
3,Stacking Super Heavy Aft Section https://t.co/...
4,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...
5,@tobyliiiiiiiii @f_rmnn Cool


In [334]:
df3.input = df3.input.apply(stemming)

In [335]:
df3.input

0                                             billym k
1              top starbas high bay http co krtjbrwrqw
2    rt cleantechnica jay leno break world record t...
3     stack super heavi aft section http co itydacq hm
4    mileycyru h x r got phone nuthin could sorri babe
5                           tobyliiiiiiiii f rmnn cool
Name: input, dtype: object

In [336]:
x1= vect.fit_transform(df3.input)

In [372]:
pred3 = model.predict(x1)
pred3=np.argmax(pred3,axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


In [373]:
pred3

array([1, 1, 1, 1, 2, 1], dtype=int64)